In [1]:
# Конфигурация: использовать ТОЛЬКО PyTorch (не TensorFlow)
import os
os.environ['USE_TF'] = '0'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

print('✓ Настроен PyTorch-only режим')

✓ Настроен PyTorch-only режим


In [2]:
import sys
import os
import pickle
import numpy as np
import random
import shutil
from tqdm.auto import tqdm
import torch

# Добавление src в путь для импорта модулей проекта
sys.path.append('../../')

# Импорт необходимых компонентов системы
from src.kg_model.embeddings_model import EmbeddingsModel, EmbeddingsModelConfig, EmbedderModelConfig
from src.db_drivers.vector_driver import VectorDriverConfig, VectorDBConnectionConfig, VectorDBInstance
from src.utils.data_structs import TripletCreator, NodeCreator, NodeType, RelationCreator, RelationType
from src.utils import Logger

/Users/nmuravya/.pyenv/versions/3.11.7/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.30.2 at schema.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/nmuravya/.pyenv/versions/3.11.7/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.30.2 at common.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/nmuravya/.pyenv/versions/3.11.7/lib/python3.11/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.30.2 at milvus.proto. Please update the gencode to avoid compatibility violations in the next runtime 

## 1. Загрузка и подготовка данных Wikidata

Данные WikidataBig хранятся в формате pickle и содержат:
- `ent_id`: словарь соответствия Q-идентификаторов (строк) числовым ID.
- `rel_id`: словарь соответствия P-идентификаторов (отношений) числовым ID.
- `ts_id`: словарь временных меток.
- `test.pickle` / `valid.pickle`: массивы триплетов с временными интервалами.

In [3]:
DATA_PATH = '../../wikidata_big/kg/tkbc_processed_data/wikidata_big/'

def load_wikidata_mapping(file_name):
    with open(os.path.join(DATA_PATH, file_name), 'rb') as f:
        return pickle.load(f)

print("Загрузка словарей соответствия...")
ent_to_id = load_wikidata_mapping('ent_id')
rel_to_id = load_wikidata_mapping('rel_id')
ts_to_id = load_wikidata_mapping('ts_id')

# Создаем обратные словари для восстановления строк по ID
id_to_ent = {v: k for k, v in ent_to_id.items()}
id_to_rel = {v: k for k, v in rel_to_id.items()}
id_to_ts = {v: str(k) for k, v in ts_to_id.items()}

print(f"Загружено сущностей: {len(ent_to_id)}")
print(f"Загружено отношений: {len(rel_to_id)}")
print(f"Загружено временных меток: {len(ts_to_id)}")

Загрузка словарей соответствия...
Загружено сущностей: 125726
Загружено отношений: 203
Загружено временных меток: 9621


In [4]:
def load_text_mapping(file_path):
    mapping = {}
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split('	')
                if len(parts) >= 2:
                    mapping[parts[0]] = parts[1]
    return mapping

ENT_TEXT_PATH = '../../wikidata_big/kg/wd_id2entity_text.txt'
REL_TEXT_PATH = '../../wikidata_big/kg/wd_id2relation_text.txt'

ent_labels = load_text_mapping(ENT_TEXT_PATH)
rel_labels = load_text_mapping(REL_TEXT_PATH)
print(f"Loaded {len(ent_labels)} entity labels and {len(rel_labels)} relation labels")

def get_label(id_str, mapping):
    return mapping.get(id_str, id_str)

def format_ts(ts_tuple):
    y, m, d = ts_tuple
    if m == 0 and d == 0: return str(y)
    return f"{y}-{m:02d}-{d:02d}"

ts_to_id_inv = {v: k for k, v in ts_to_id.items()}


Loaded 125725 entity labels and 203 relation labels


In [5]:
print("Загрузка тестовых данных...")
with open(os.path.join(DATA_PATH, 'test.pickle'), 'rb') as f:
    test_data = pickle.load(f) # Формат: (s, r, o, start_t, end_t)

print(f"Количество тестовых квадруплетов: {len(test_data)}")

Загрузка тестовых данных...
Количество тестовых квадруплетов: 4995


## 2. Конвертация в формат системы (Triplet)

Мы преобразуем числовые данные Wikidata в объекты `Triplet`, используя нашу обновленную структуру с поддержкой времени.

In [6]:
def convert_to_triplets(data_subset, sample_limit=4995):
    subset = data_subset[:sample_limit]
    converted = []
    
    for row in tqdm(subset, desc="Конвертация"):
        s_id = id_to_ent[row[0]]
        r_id = id_to_rel[row[1]]
        o_id = id_to_ent[row[2]]
        t_tuple = ts_to_id_inv[row[3]]
        
        s_name = get_label(s_id, ent_labels)
        r_name = get_label(r_id, rel_labels)
        o_name = get_label(o_id, ent_labels)
        t_name = format_ts(t_tuple)
        
        s_node = NodeCreator.create(NodeType.object, s_name, prop={'t_id': s_id})
        r_rel = RelationCreator.create(RelationType.simple, r_name, prop={'t_id': r_id})
        o_node = NodeCreator.create(NodeType.object, o_name, prop={'t_id': o_id})
        t_node = NodeCreator.create(NodeType.time, t_name, prop={'t_id': str(row[3])})
        
        triplet = TripletCreator.create(s_node, r_rel, o_node, time=t_node)
        converted.append(triplet)
        
    return converted

test_triplets = convert_to_triplets(test_data)
print(f"Пример: {test_triplets[0].stringified}")


Конвертация:   0%|          | 0/4995 [00:00<?, ?it/s]

Пример: 1918: Walter Judd military branch United States Army


## 3. Инициализация модели и расчет метрик

Мы инициализируем `EmbeddingsModel` и проводим поиск по векторизованному представлению запроса `(s, p, t)` для нахождения верного `o`.

In [7]:
# Конфигурация векторного хранилища (тестовая)
NODES_DB_PATH = '../../data/graph_structures/vectorized_nodes/wikidata_test'
TRIPLETS_DB_PATH = '../../data/graph_structures/vectorized_triplets/wikidata_test'
# Используем HuggingFace Hub если локальная модель отсутствует
EMBEDDER_PATH = '../../models/wikidata_finetuned'
if not os.path.exists(EMBEDDER_PATH):
    EMBEDDER_PATH = 'intfloat/multilingual-e5-small'
# Автоопределение устройства (CUDA/MPS/CPU)
from src.utils.device_utils import get_device
DEVICE = get_device()

# Очистка предыдущих тестов
for path in [NODES_DB_PATH, TRIPLETS_DB_PATH]:
    if os.path.exists(path): shutil.rmtree(path)

config = EmbeddingsModelConfig(
    nodesdb_driver_config=VectorDriverConfig(db_config=VectorDBConnectionConfig(conn={"path": NODES_DB_PATH}, need_to_clear=True)),
    tripletsdb_driver_config=VectorDriverConfig(db_config=VectorDBConnectionConfig(conn={"path": TRIPLETS_DB_PATH}, need_to_clear=True)),
    embedder_config=EmbedderModelConfig(model_name_or_path=EMBEDDER_PATH, device=DEVICE)
)

model = EmbeddingsModel(config)
# model.embedder.init_model()

# Индексация данных
print("Индексация тестовых триплетов в векторную БД...")
model.create_triplets(test_triplets)

✓ Используется Apple Silicon GPU (MPS)


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

Индексация тестовых триплетов в векторную БД...


100%|██████████| 40/40 [00:27<00:00,  1.47it/s]


{'nodes': {'326527366ba493d1e2d0ecb6a8f07260',
  '3b68abe12cb49c6290af0614171d8096',
  '8e03abada50dde8db74ea2c7ec948f3e',
  '6b861ce12cda50c3abdbb7e4e5988a6f',
  'e2c5e050c4744830f3bbb41f6cfebd62',
  'b97b01ad1a9155618ffb7cc15c2b36c0',
  '1ba8bd4ecf0a0e8d2e48cb5ab8f3c886',
  '5c5c83e9af8b5aa29c6afd70674ab1ef',
  '78c1ef521faffdc5adca6509d6bc0348',
  '7217a2aa3d79b70473d015b09f46a8c3',
  '8ca370437d49e8bfc6f4772bc738bba7',
  'd63a8dba97f3bf5b741dd86f91f98e8e',
  '2c326bd059f56cdde4c75d4be6a9f872',
  'e10f0cc24d0ae601d632f97c67aa92dc',
  '8b55732d5d34f5d76cbbb08430d10114',
  'd41e0b4a93bdd5ea4eae40dda13a71fa',
  'f367d0feff412dd9565635c3096e28b8',
  '7cd47f6d620e861cc8209c5e4fd690dc',
  '46ba99cd0ddf05df45e1b693106d75ee',
  '32d4613ce62e94c1c405970790823e23',
  'b6c7d0236d9a4e9f993955952dac1a21',
  '3975ea99aa7284f3b0da4d2bbfa932a2',
  '5b6eb519d7cf99b693419e784705ae19',
  'fad4c5c6cbc1798f457deecdb3bfbf88',
  '9674e57e11c4f606fe5a416a8512a658',
  'a0103ed214d2bcf1a33501d6757a877a',
  '

## 4. Расчет MRR и Hits@K

**Метрики:**
- **Hits@K**: Доля случаев, когда правильный ответ находится в топ-K результатах.
- **MRR (Mean Reciprocal Rank)**: Среднее значение величины, обратной рангу правильного ответа.

In [8]:

def evaluate_comprehensive(model, triplets, k_values=[1, 5, 10, 100]):
    metrics = {
        'object': {'hits': {k: 0 for k in k_values}, 'mrr': 0, 'count': 0},
        'subject': {'hits': {k: 0 for k in k_values}, 'mrr': 0, 'count': 0},
        'time': {'hits': {k: 0 for k in k_values}, 'mrr': 0, 'count': 0}
    }

    print(f"Starting comprehensive evaluation on {len(triplets)} triplets...")

    for triplet in tqdm(triplets, desc="Оценка"):
        t_str = triplet.time.name if triplet.time else ""
        s_str = triplet.start_node.name
        r_str = triplet.relation.name
        o_str = triplet.end_node.name
        
        # 1. Predict Object: (S, R, T) -> O
        query_o = f"{t_str}: {s_str} {r_str}"
        _update_metrics(model, query_o, triplet.end_node.id, metrics['object'], k_values)
        
        # 2. Predict Subject: (O, R, T) -> S
        query_s = f"{t_str}: {r_str} {o_str}"
        _update_metrics(model, query_s, triplet.start_node.id, metrics['subject'], k_values)
        
        # 3. Predict Time: (S, R, O) -> T
        if triplet.time:
            query_t = f"When did {s_str} {r_str} {o_str}?"
            _update_metrics(model, query_t, triplet.time.id, metrics['time'], k_values)

    # Normalize
    results = {}
    for m_type, data in metrics.items():
        count = data['count']
        if count > 0:
            res = {'mrr': data['mrr'] / count}
            res.update({f'hits@{k}': data['hits'][k] / count for k in k_values})
            results[m_type] = res
        else:
            results[m_type] = None
            
    return results

def _update_metrics(model, query_text, gold_id, metric_dict, k_values, is_query=True):
    try:
        if is_query:
            q_emb = model.embedder.encode_queries([query_text])[0]
        else:
            q_emb = model.embedder.encode_passages([query_text])[0]
            
        query_inst = VectorDBInstance(id='q', document=query_text, embedding=q_emb)
        
        results = model.vectordbs['nodes'].retrieve([query_inst], n_results=100, includes=[])
        candidates = results[0]
        
        rank = None
        for i, (dist, inst) in enumerate(candidates):
            if inst.id == gold_id:
                rank = i + 1
                break
                
        if rank is not None:
            metric_dict['mrr'] += 1.0 / rank
            for k in k_values:
                if rank <= k: metric_dict['hits'][k] += 1
                
        metric_dict['count'] += 1
    except Exception as e:
        pass


In [9]:

results = evaluate_comprehensive(model, test_triplets)

print("\n=== РЕЗУЛЬТАТЫ WIKIDATA BIG ===")
for task, res in results.items():
    if res:
        print(f"\n[{task.upper()} PREDICTION]")
        print(f"MRR: {res['mrr']}")
        for k in [1, 5, 10, 100]:
            print(f"Hits@{k}: {res[f'hits@{k}']}")
    else:
        print(f"\n[{task.upper()}] No data")


Starting comprehensive evaluation on 4995 triplets...


Оценка:   0%|          | 0/4995 [00:00<?, ?it/s]


=== РЕЗУЛЬТАТЫ WIKIDATA BIG ===

[OBJECT PREDICTION]
MRR: 0.010359665003682303
Hits@1: 0.0002002002002002002
Hits@5: 0.017417417417417418
Hits@10: 0.02882882882882883
Hits@100: 0.11411411411411411

[SUBJECT PREDICTION]
MRR: 0.004912023847163837
Hits@1: 0.0004004004004004004
Hits@5: 0.007807807807807808
Hits@10: 0.012812812812812813
Hits@100: 0.05405405405405406

[TIME PREDICTION]
MRR: 0.014473251810640998
Hits@1: 0.0006006006006006006
Hits@5: 0.015415415415415416
Hits@10: 0.02822822822822823
Hits@100: 0.2764764764764765
